# Lab 2 - Statistical Inference

## Preparation Phase

In [1]:
library("readxl")

Warning message:
"package 'readxl' was built under R version 4.4.2"


In [2]:
df <- read_excel("ElephantShrew.xlsx")
dim(df)
summary(df)

[1] 75  4

     Sex               BodyMass       Captured           EarLength    
 Length:75          Min.   :36.30   Length:75          Min.   :21.60  
 Class :character   1st Qu.:45.70   Class :character   1st Qu.:25.50  
 Mode  :character   Median :49.70   Mode  :character   Median :30.50  
                    Mean   :49.52                      Mean   :30.52  
                    3rd Qu.:53.50                      3rd Qu.:35.15  
                    Max.   :67.40                      Max.   :40.00  

There 4 variables in the dataset:
- Sex: nominal, with two values
- BodyMass: ratio
- Captured: nominal, with two values
- EarLength: ratio

The measurement level is important because it helps analysts to find the right way to do statistcal tests, descriptive analysis and mathematical processing, which would influence the validity of data analysis and conclusions.

## Question 4

We are gonna calculate the 95% CI for the BodyMass:
- the population sd is unknown to us, thus the t-statistics needs to be calculated, instead of z-statistics.
- we don't know whether the bodymass of the population is noamally distributed, but fortunately we got a large enough sample size (n = 75) to conduct the estimation (so that the distribution of the sample mean would be roughtly normal).
- next we need to calculate the t-statistics, given the df = 74 and alpha = 0.05.

In [3]:
upper_cv <- qt(p = 0.975,df = 74)
se <- sd(df$BodyMass) / sqrt(75)
upper_limit <- mean(df$BodyMass) + upper_cv * se
lower_limit <- mean(df$BodyMass) - upper_cv * se

print(upper_limit)
print(lower_limit)

[1] 51.05747


[1] 47.97986


## Question 5 and 6

We are 95% sure that the parameter of the population (bodymass) is in this interval, which means if we repeatedly sampling and calculate this way, around 95% of the CIs will contain the real parameter.

If we increase the confidence level (e.g., from 95% to 99%), than the CI we get will be wider, because as the confidence level going up, the absolute value of critical value (i.e., t_{frac{\alpha}{2}}, the reason is trivial: we should make the distance between two critical values larger to make the square under the density function larger, so that the confidence level will go up) will go up either, making the upper bound larger and lower bound smaller.

## Question 7

In [4]:
upper_cv <- qt(p = 0.995,df = 74)
se <- sd(df$BodyMass) / sqrt(75)
upper_limit <- mean(df$BodyMass) + upper_cv * se
lower_limit <- mean(df$BodyMass) - upper_cv * se

print(upper_limit)
print(lower_limit)

[1] 51.56051
[1] 47.47682


## Question 8

We are gonna do this one-sample test:
- following the same reason, we will conduct the t-test instead of z-test, with a significant level of 0.05.
- H0: the average bodymass is no greater than 48 (miu <= 48)
- H1: the average bodymass is greater than 48 (miu > 48)
- thus the test is one-sided, we can use the argument "alternative", letting it be "greater".

In [5]:
# First we mannully do the t-test
t_statistics <- (mean(df$BodyMass) - 48) / se
p_value <- 1 - pt(q = t_statistics,df = 74)
cv <- qt(0.95,df = 74)

print(t_statistics)
print(cv)
print(p_value)

[1] 1.966465
[1] 1.665707
[1] 0.02649777


As can be seen above, the t_statistics is larger than the critical value, meaning that the value is in the reject region, and the p value is smaller than the sig level we set before. Therefore we can reasonably reject H0 and say we are 95% sure that the bodymass of the population is greater than 48.

Then we can use the built-in function to validate our result:

In [6]:
t.test(x = df$BodyMass, mu = 48, alternative = "greater")


	One Sample t-test

data:  df$BodyMass
t = 1.9665, df = 74, p-value = 0.0265
alternative hypothesis: true mean is greater than 48
95 percent confidence interval:
 48.23227      Inf
sample estimates:
mean of x 
 49.51867 


## Question 9

In [7]:
var.test(BodyMass ~ Sex, data = df)


	F test to compare two variances

data:  BodyMass by Sex
F = 1.1105, num df = 33, denom df = 40, p-value = 0.7459
alternative hypothesis: true ratio of variances is not equal to 1
95 percent confidence interval:
 0.5787515 2.1764584
sample estimates:
ratio of variances 
           1.11049 


In [8]:
t.test(BodyMass ~ Sex, data = df, alternative = "two.sided")
t.test(BodyMass ~ Sex, data = df, var.equal = TRUE, alternative = "two.sided")


	Welch Two Sample t-test

data:  BodyMass by Sex
t = 8.3557, df = 68.962, p-value = 4.486e-12
alternative hypothesis: true difference in means between group Female and group Male is not equal to 0
95 percent confidence interval:
  7.120925 11.587684
sample estimates:
mean in group Female   mean in group Male 
            54.63235             45.27805 



	Two Sample t-test

data:  BodyMass by Sex
t = 8.3972, df = 73, p-value = 2.524e-12
alternative hypothesis: true difference in means between group Female and group Male is not equal to 0
95 percent confidence interval:
  7.134153 11.574456
sample estimates:
mean in group Female   mean in group Male 
            54.63235             45.27805 


Yes there is a significant difference between bodymass by gender. And the bodymass of female is probably larger than that of male in the population.

## Question 10

In [9]:
contingency_table <- table(df$Captured, df$Sex)
print(contingency_table)

       
        Female Male
  Day       14   22
  Night     20   19


In [10]:
# no need to do the correction adjustment, there's no unit smaller than 5
chisq.test(contingency_table, correct = FALSE) 
# or, we can equally run another command
chisq.test(df$Sex, df$Captured, correct = FALSE)



	Pearson's Chi-squared test

data:  contingency_table
X-squared = 1.1602, df = 1, p-value = 0.2814



	Pearson's Chi-squared test

data:  df$Sex and df$Captured
X-squared = 1.1602, df = 1, p-value = 0.2814


p-value is 0.2814, which is larger than 0.05, so we cannot reject the null hypothesisi, which is that there isn't an association between Captured and Sex.

Next we are gonna to do the chisq test manually:

In [11]:
mat <- matrix(contingency_table, nrow = 2, byrow = TRUE)

chi_statistics <- 0

for (i in 1:2){
    for (j in 1:2){
        row_sum <- sum(mat[i, ])
        col_sum <- sum(mat[, j])
        eij <- row_sum * col_sum / sum(contingency_table)
        
        temp <- (mat[i, j] - eij) ** 2 / eij
        chi_statistics = chi_statistics + temp
    }
}

print(chi_statistics)

[1] 1.160192


We got the same chi-statistics as that generated by built-in function.